In [1]:
import pickle, glob, sys, os, warnings, csv

import numpy as np

from datetime import datetime
from time import time

from scipy.stats import gamma as _Ga
from scipy.stats import vonmises as _Vm
from scipy.stats import beta as _Be
from scipy.stats import multivariate_normal, norm

# Import All custom functions
from detection_clustering_utils import *
from feature_extraction_utils import *
from cloud_velocity_vector_utils import *
from image_processing_utils import *
from cloud_segmentation_utils import *
from wind_velocity_field_utils import *
from lib_motion_vector import _farneback, _lucas_kanade, _pyramidal_weighted_lucas_kanade, _weighted_lucas_kanade
from utils import *
# Import frame-Class, models, and paramters
from config import _frame
# Import functions for a single layer wind velocity field
from extra import *
from files import *

# Do not display warnings in the output file
warnings.filterwarnings('ignore')

# Compute statisticd of features (Modify this Code)
def _get_stats(K_0_, K_1_, K_2_, K_3_, H_2_, N_2_, M_, D_, V_, W_):
    # Computed Weighted mean an variance
    def __stats(X_, W_):
        # Vectors Definitio
        x_ = X_.flatten()
        w_ = W_.flatten()
        o_ = np.ones(x_.shape)
        # Weights Normalization
        w_ /= w_.sum()
        # Weighted Sample Mean
        mu = (w_.T @ x_) / (o_.T @ w_)
        z_ = (x_ - mu)
        # Weighted Sample Variance
        s = ( z_.T @ (np.eye(w_.shape[0])*w_) @ z_ ) / (o_.T @ w_)
        # Weighted Sample Skewness
        g = ( w_.T @ z_**3 / (o_.T @ w_) ) / ( w_.T @ z_**2 / (o_.T @ w_) )**(3/2)
        # Weighted Sample Kurtosis
        k = ( w_.T @ z_**4 / (o_.T @ w_) ) / ( w_.T @ z_**2 / (o_.T @ w_) )**2
        # Make sure there are no Nans
        return np.nan_to_num(mu), np.nan_to_num(s), np.nan_to_num(g), np.nan_to_num(k)
    # Compute Cloud Features Statistics
    m_m, s_m, g_m, k_m = __stats(M_, W_)
    m_d, s_d, g_d, k_d = __stats(D_, W_)
    m_v, s_v, g_v, k_v = __stats(V_, W_)
    m_h, s_h, g_h, k_h = __stats(H_2_, W_)
    # Compute Clouds Temperature Statistics
    m_k0, s_k0, g_k0, k_k0 = __stats(K_0_, W_)
    m_k1, s_k1, g_k1, k_k1 = __stats(K_1_, W_)
    m_k2, s_k2, g_k2, k_k2 = __stats(K_2_, W_)
    m_k3, s_k3, g_k3, k_k3 = __stats(K_3_, W_)
    m_n,  s_n,  g_n,  k_n  = __stats(N_2_, W_)
    # Get Features Statistics
    m_feat_ = np.asarray([m_m, m_d, m_v, m_h])
    s_feat_ = np.asarray([s_m, s_d, s_v, s_h])
    g_feat_ = np.asarray([g_m, g_d, g_v, g_h])
    k_feat_ = np.asarray([k_m, k_d, k_v, k_h])
    # Get Temperature Statistics
    m_temp_ = np.asarray([m_k0, m_k1, m_k2, m_k3, m_n])
    s_temp_ = np.asarray([s_k0, s_k1, s_k2, s_k3, s_n])
    g_temp_ = np.asarray([g_k0, g_k1, g_k2, g_k3, g_n])
    k_temp_ = np.asarray([k_k0, k_k1, k_k2, k_k3, k_n])
    return m_feat_, s_feat_, g_feat_, k_feat_, m_temp_, s_temp_, g_temp_, k_temp_

def _autoregressive_param_v0(Mu_, Cov_, m_hat_, C_hat_, e_, error, k, lag = 10):
    #if k == 0:
    mu_e  = np.nan_to_num(np.mean(e_), nan = 0.)
    std_e = np.nan_to_num(np.std(e_), nan = 0.)
    prob  = np.nan_to_num(norm(mu_e, std_e).pdf(error), nan = 1.)
    # Save Parameters if there is not risen flag
    if np.isnan(m_hat_).any() or np.isnan(C_hat_).any() or (prob < 0.03):
        print('Flag!')
    else:
        # Keep the desire number of lags on the list by removing the last and aadding at the bigging
        if len(Mu_[k]) == lag:
            Mu_[k].pop(0)
            Cov_[k].pop(0)
        if len(e_) == lag and (k == 0):
            e_.pop(0)
        # Keep adding until we have the desired number of lag time stamps
        Mu_[k].append(m_hat_)
        Cov_[k].append(C_hat_)
    if k == 0: e_.append(error)
    # Do I want to average Parameters in last lags?
    m_hat_ = Mu_[k][-1]
    C_hat_ = Cov_[k][-1]
    return Mu_, Cov_, m_hat_, C_hat_

def _autoregressive_param_v1(Mu_, Cov_, m_hat_, C_hat_, e_, error, k, lag = 10):
    mu_e  = np.nan_to_num(np.mean(e_), nan = 0.)
    std_e = np.nan_to_num(np.std(e_), nan = 0.)
    _N    = norm(mu_e, std_e)
    ci_   = _N.interval(0.95)
    prob  = np.nan_to_num(_N.pdf(error), nan = 1.)
    if k == 3:
        e_.append(error)
    if prob < 0.025:
        pass
    else:
        # Keep the desire number of lags on the list by removing the last and aadding at the bigging
        if len(Mu_[k]) == lag:
            Mu_[k].pop(0)
            Cov_[k].pop(0)
        if len(e_) == lag and (k == 3):
            e_.pop(0)
        # Keep adding until we have the desired number of lag time stamps
        Mu_[k].append(m_hat_)
        Cov_[k].append(C_hat_)
    # Do I want to average Parameters in last lags?
    m_hat_ = np.mean(np.stack(Mu_[k]), axis = 0)
    C_hat_ = np.mean(np.stack(Cov_[k]), axis = 0)
    return Mu_, Cov_, m_hat_, C_hat_


def _get_covariates(X_, lag, index):
    idx, t, py_, gsi_, csi_, el_, az_ = X_[index][0]
    ref_  = np.asarray([i, t])
    angl_ = np.asarray([el_, az_])
    auto_ = [[csi_, gsi_, py_],]
    for l in range(1, lag):
        idx, t, py_, gsi_, csi_, el_, az_  = X_[index - l][0]
        auto_.append([csi_, gsi_, py_])
    return ref_, angl_, np.stack(auto_)

def _get_predictors(X_, horion, index):
    idx, t, py_, gsi_, csi_, el_, az_ = X_[index + horion][0]
    return np.asarray([csi_, gsi_, py_])

In [2]:
# Finding index of pixels expecting to intercept the Sun for each horizon (k)
def _sun_intercepting_line(Phi_, Psi_, x_0_, A_0_):
    # Selecting pixels by streamlines and potential lines that intercept the Sun
    def __select_intercepting_potential(Psi_0):
        return Psi_ > Psi_0
    # Connected components form the Sun streamline alon equipotential streamlines
    def __select_intercepting_streamline(Phi_, Psi_, x_0, y_0, Phi_0, idx_potential_):
        # Finding connected pixel
        def ___next_pixel_coordiantes(Phi_, Psi_, i, j, idx_):
            Psi_0 = Psi_[i,j]
            # Defining errors matrix
            E_ = np.zeros((3, 3))
            #W_ = np.zeros((3, 3))
            # loop over nerbouring pixels
            for k in [-1, 0, 1]:
                for m in [-1, 0, 1]:
                    #C_ = idx_[i + k - 1: i + k + 2, j + m - 1: j + m + 2]
                    if k == 0 and m == 0:
#                         E_[1 + k, 1 + m] = - np.inf
#                     else:
#                         E_[1 + k, 1 + m] = Phi_[i + k, j + m] #np.sqrt((Phi_0 - Phi_[i + k, j + m])**2)
#                         #W_[1 + k, 1 + m] = Psi_[i + k, j + m] - Psi_
                        E_[1 + k, 1 + m] = np.inf
                    else:
                        E_[1 + k, 1 + m] = (Phi_0 - Phi_[i + k, j + m])**2
                        W_[1 + k, 1 + m] = Psi_[i + k, j + m] - Psi_0
#             # Unravel error matrix coordiantes of min value error
#             #E_[W_ > 0.] = - np.inf
#             k_, m_ = np.where(E_ == E_.max())
#             # Updating new streamline pixel coordiantes
#             i_new, j_new = i + k_[0] - 1, j + m_[0] - 
            # Unravel error matrix coordiantes of min value error
            idx_ = W_ <= 0.
            k_, m_ = np.where(E_ == E_[idx_].min())
            # Updating new streamline pixel coordiantes
            i_new, j_new = i + k_[0] - 1, j + m_[0] - 1
            return i_new, j_new
        # Find the trasition on the connected components of the streamlines
        def __get_transitions_index(i_0, j_0, i, j):
            a, b = 0, 0
            if i_0 != i: a = 1
            if j_0 != j: b = 1
            return [a, b]
        # Variables initialization
        i_0, j_0 = y_0, x_0
        idx_1_   = idx_potential_.copy()
        idx_2_   = np.zeros((N_y, N_x), dtype = int)
        idx_3_   = []
        # Position Initialization
        count = 1
        
#         plt.figure()
#         plt.imshow(Phi_)
#         plt.colorbar()
#         plt.show()
        
#         plt.figure()
#         plt.imshow(Psi_)
#         plt.colorbar()
#         plt.show()
        
        X_ = np.sqrt((Phi_ - Phi_[i_0, j_0])**2)
        X_ = X_.max() - X_
#         plt.figure()
#         plt.imshow(X_)
#         plt.colorbar()
#         plt.show()
        
        Z_ = X_ + 1j*Psi_
        
#         plt.figure()
#         plt.imshow(np.absolute(Z_)**2)
#         plt.colorbar()
#         plt.show()

        W_ = 1. - 1.*idx_potential_

#         plt.figure()
#         plt.imshow(W_)
#         plt.colorbar()
#         plt.show()

        X_ = W_*np.absolute(Z_)**2 
        X_ = X_/X_.sum()

#         plt.figure()
#         plt.imshow(X_)
#         plt.colorbar()
#         plt.show()
        
        Phi_  = X_.copy()
        Phi_0 = Phi_[i_0, j_0]

        
        #idx_1_[i_0, j_0] = True
        #idx_2_[i_0, j_0] = count
        # Loop to the edge of the frame
        while True:
            count += 1
            # Finding next pixel on the streamline
            i, j = ___next_pixel_coordiantes(Phi_, Psi_, i_0, j_0, idx_1_)
            # Updating positon
            idx_1_[i, j] = True
            idx_2_[i, j] = count
            Phi_[i, j]   = 0.
            idx_3_.append(__get_transitions_index(i_0, j_0, i, j))
            i_0 = i.copy()
            j_0 = j.copy()
            
#             plt.figure()
#             plt.imshow(idx_2_)
#             plt.show()


            # Finding the edge
            if (i >= 59 or i <= 0) or (j >= 79 or j <= 0):
                break
        
        Z_potential_  = np.absolute(Z_)**2 
        Z_potential_ /= Z_potential_.sum()   
        
        return idx_2_, idx_3_, Z_potential_
    # Sun position on integer value to use it as indexing value
    x_0 = int(np.around(x_0_[0]))
    y_0 = int(np.around(x_0_[1]))
    # Sun-interceptig streamline
    phi_0 = Phi_[y_0, x_0]
    psi_0 = Psi_[y_0, x_0]
    # Selecting Sun intercepting potential line and streamline index
    idx_potential_line_ = __select_intercepting_potential(psi_0)  
    idx_streamline_, idx_transitions_, Z_potential_ = __select_intercepting_streamline(Phi_, Psi_, x_0, y_0, phi_0, idx_potential_line_)
    return idx_potential_line_, idx_streamline_, idx_transitions_, Z_potential_

# Finding index of pixels expecting to intercept the Sun for each horizon (k)
def _sun_intersecting_dist(UV_hat_, XYZ_, dXYZ_, x_0_, idx_streamline_, idx_transitions_, height, error_, g):
    # Estimate circule center for selection according to estimate arrivale time of a pixel
    def __estimate_interception(XYZ_, dXYZ_, U_, V_, e_x, e_y, phi_, transitions_, idx_phi_, g):
    
        # For an unbiased estimator, the RMSD is the square root of the variance, known as the standard deviation.
        def ___compute_time_uncertanty(dx_, dy_, u_, v_, transitions_, e_x, e_y):
            # Variables Initialization
            t_bar_  = np.zeros((len(transitions_),))
            s_t_bar_ = np.zeros((len(transitions_),))
            # Calculate the distance components for each streamline pixel
            for i, transition_ in zip(range(len(transitions_)), transitions_):
                a, b = transition_
                # Draw samples
                e_x_ = np.random.normal(loc = 0, scale = e_x, size = 1000)
                e_y_ = np.random.normal(loc = 0, scale = e_y, size = 1000)
                # Calculate the velocity components for each streamline pixel
                u_samples_ = u_[i] + e_x_
                v_samples_ = v_[i] + e_y_
                w_samples_ = np.sqrt(u_samples_**2 + v_samples_**2) + 1e-25
                #r_ = (np.sqrt(u_[i]**2 + v_[i]**2) + 1e-25)/(np.sqrt(a*u_[i]**2 + b*v_[i]**2) + 1e-25)
                #dz = r_*np.sqrt(a*dx_[i]**2 + b*dy_[i]**2)                
                # Calculate the distance to traverse in a pixel
                dz = np.sqrt(a*dx_[i]**2 + b*dy_[i]**2)
                # Compute Time
                T_samples_ = dz/w_samples_
                plt.figure()
                plt.hits(T_samples_)
                plt.show()
                # Inference distribution from samples
                t_bar_[i]   = np.mean(T_samples_)
                s_t_bar_[i] = np.var(T_samples_)
            # Integrating and solving for time for each component
            t_bar_   = np.cumsum(t_bar_)
            s_t_bar_ = np.cumsum(s_t_bar_)
            return t_bar_, s_t_bar_
        
        # Compute propagate the undertanty in the velocity vectors to the space
        def ___compute_space_uncertanty(g, u, v, e_x, e_y):
            # Draw samples
            e_x_ = np.random.normal(loc = 0, scale = e_x, size = 1000)
            e_y_ = np.random.normal(loc = 0, scale = e_y, size = 1000)
            # Calculate the velocity components for each streamline pixel
            u_samples = u + e_x_
            v_samples = v + e_y_
            w_samples = np.sqrt(u_samples**2 + v_samples**2) + 1e-25
            # Compute Distance
            X_samples_ = g*w_samples
            plt.figure()
            plt.hits(X_samples_)
            plt.show()
            # Inference distribution from samples
            s_x_bar    = np.var(X_samples_)
            return s_x_bar
                
        # Initialize variables, identify pixels on the streamline, and proximity sorting index definition
        idx_ = np.argsort(phi_[idx_phi_] - 1)
        # Velocity compoenents
        u_ = U_[idx_phi_][idx_]
        v_ = V_[idx_phi_][idx_]
        # Numerical differenciation on a non-linear frame-metric
        dx_ = dXYZ_[idx_phi_, 0][idx_]
        dy_ = dXYZ_[idx_phi_, 1][idx_]
        # Compute propagate the undertanty in the velocity vectors to the time
        t_bar_, s_t_bar_ = ___compute_time_uncertanty(dx_, dy_, u_, v_, transitions_, e_x, e_y)
        # Arrival Time Distribution
        z_ = np.exp( - .5*(t_bar_ - g)**2/(s_t_bar_))
        o_ = np.ones(z_.shape)
        # Distances in non-linear coordinates
        x_ = XYZ_[idx_phi_, 0][idx_]
        y_ = XYZ_[idx_phi_, 1][idx_]
        # Normal Distribution Parameters Approximation
        x_hat = (z_.T @ x_) / (o_.T @ z_)
        y_hat = (z_.T @ y_) / (o_.T @ z_)
        u_hat = (z_.T @ u_) / (o_.T @ z_)
        v_hat = (z_.T @ v_) / (o_.T @ z_)
        # Compute propagate the undertanty in the velocity vectors to the space
        s_x_bar = ___compute_space_uncertanty(g, u_hat, v_hat, e_x, e_y)
        return np.asarray([x_hat, y_hat]), np.eye(2)*s_x_bar
    # Get uncertanties on the velocity fectors estimation
    e_z, e_x, e_y = error_
    # Saparete Velocity components
    ifps   = 15.
    U_hat_ = UV_hat_[..., 0]*ifps
    V_hat_ = UV_hat_[..., 1]*ifps
    e_x   *= ifps
    e_y   *= ifps
    e_z   *= ifps
    # Time estimation for intercepting the Sun for each pixel on the streamline
    return __estimate_interception(XYZ_, dXYZ_, U_hat_, V_hat_, e_x, e_y, idx_streamline_, idx_transitions_, 
                                   idx_streamline_ > 0, g)


In [3]:
import matplotlib.pyplot as plt 

# Get Files in Directory
name        = r'/users/terren/wheeler-scratch/data_feature_extraction_v2/*'
file_names_ = sorted(glob.glob(name))
print(file_names_)

# Configure Selection of Features
N_x = 80
N_y = 60
lag = 6
g_  = [4, 8, 16, 32]

# Loop over files
for i in range(1, len(file_names_)):
    
    print(i, file_names_[i])

    with open(file_names_[i], 'rb') as file:
        X_ = pickle.load(file)

    Mu_   = [[], [], [], []]
    Cov_  = [[], [], [], []]
    e_    = []
    data_ = [['References', 'Temperatue Image', 'Likelihood Pixels',
             '[k + 2, k + 4, k + 8, k + 16] x 3 Predictors: CSI | GSI | PYR', '2 Angles', '[k, ..., k - L] x 3 AR Dataset: CSI | GSI | PYR',
             ['[k + 2, k + 4, k + 8, k + 16] x 4 Features Stats: E(M) | E(D) | E(V) | E(H)',
              '[k + 2, k + 4, k + 8, k + 16] x 4 Features Stats: VAR(M) | VAR(D) | VAR(V) | VAR(H)',
              '[k + 2, k + 4, k + 8, k + 16] x 5 Temp. Stats: E(K_0) | E(K_1) | E(K_2) | E(K_3) | E(I_nrm)',
              '[k + 2, k + 4, k + 8, k + 16] x 5 Temp. Stats: VAR(K_0) | VAR(K_1) | VAR(K_2) | VAR(K_3) | VAR(I_nrm)']]]
    # Loop over samples in a file
    for j in range(lag, len(X_) - g_[-1]):
        print(j, len(X_), len(X_[j]))
        try:
            # Extract Radiometry Features
            K_0_, K_1_, K_2_, H_2_, N_2_, I_segm_, Z_segm_, x_0_, A_0_, T = X_[j][1]
            # Compute Relative Temperature w.r.t. the Tropopause
            K_3_ = K_2_ - T/100.
            P_   = np.zeros((N_y, N_x))
            # Initialize Variable
            M_f_ = np.zeros((4, len(g_)))
            S_f_ = np.zeros((4, len(g_)))
            G_f_ = np.zeros((4, len(g_)))
            K_f_ = np.zeros((4, len(g_)))

            M_t_ = np.zeros((5, len(g_)))
            S_t_ = np.zeros((5, len(g_)))
            G_t_ = np.zeros((5, len(g_)))
            K_t_ = np.zeros((5, len(g_)))

            # Get Sample Reference Time Series and Angles Features
            ref_, angl_, auto_ = _get_covariates(X_, lag, index = j)

            pred_ = []
            # Loop over forecating horizons
            for k in range(len(g_)):
                pred_.append( _get_predictors(X_, g_[k], index = j) )
            pred_ = np.stack(pred_)

            # Extract Features only if enough clouds were detected to estimate the wind flow
            if len(X_[j]) != 2:
                # Extract Flow Features
                height, UV_hat_, Phi_, Psi_, XYZ_, dXYZ_, WRMSE_ = X_[j][2][0]
                # Extract Cloud Velocity Vectors Features
                M_, D_, V_ = X_[j][3]
                # Adjust Prespective transformation to current height
                dXYZ_ = height*dXYZ_
                XYZ_  = height*XYZ_
                # Compute Intercepting Streamline
                idx_potential_line_, idx_streamline_, idx_transitions_, Z_potential_ = _sun_intercepting_line(Phi_, Psi_, x_0_, A_0_)

                print((idx_streamline_ > 0.).sum())

    #             plt.figure()
    #             plt.imshow(idx_streamline_)
    #             plt.show()
                # Loop over forecating horizons
                for k in range(len(g_)):

                    # Infer the Parameters of the distribution of the pixel intercepting the Sun
                    m_hat_, C_hat_ = _sun_intersecting_dist(UV_hat_, XYZ_, dXYZ_, x_0_, idx_streamline_, idx_transitions_,
                                                                height, WRMSE_, g_[k])

        #             #Mu_, Cov_, m_hat_, C_hat_ = _autoregressive_param_v0(Mu_, Cov_, m_hat_, C_hat_, e_, WMSE_ts, k)

        #             Mu_, Cov_, m_hat_, C_hat_ = _autoregressive_param_v1(Mu_, Cov_, m_hat_, C_hat_, e_, WRMSE_ts, k)

                    # Compute Spatial Probabilitiy
                    Z_sector_ = multivariate_normal(m_hat_, C_hat_).pdf(XYZ_[..., :2].reshape(N_y*N_x, 2)).reshape(N_y, N_x)**2
                    Z_sector_ /= Z_sector_.sum()
                    Z_ = Z_sector_*Z_potential_
                    Z_ /= Z_.sum()
                
                    # Compute the Probabilitiy of intercepting the Sun of each Pixel given the horion g-th
                    #W_  = Z_ > 0. #* Z_segm_ #v3
                    #W_  = Z_ v2
                    P_ += Z_*(k**2)

                plt.figure(figsize = (20, 5))
                plt.subplot(151)
                plt.imshow(K_0_)
                plt.colorbar()
                plt.subplot(152)
                plt.imshow(K_1_)
                plt.colorbar()
                plt.subplot(153)
                plt.imshow(N_2_)
                plt.colorbar()
                plt.subplot(154)
                plt.imshow(1.*idx_streamline_)
                plt.colorbar()
                plt.subplot(155)
                plt.imshow(P_)
                plt.colorbar()
                plt.show()

    #             M_f_[:, k], S_f_[:, k], G_f_[:, k], K_f_[:, k], M_t_[:, k], S_t_[:, k], G_t_[:, k], K_t_[:, k] = _get_stats(K_0_, K_1_, K_2_, K_3_,
    #                                                                                                                          H_2_, N_2_, M_, D_, V_, W_)

    #         data_.append([ref_, K_3_, P_, pred_, angl_, auto_, [M_f_, S_f_, G_f_, K_f_, M_t_, S_t_, G_t_, K_t_]])
        except:
            print('error!')
            pass

# save_path = r'/users/terren/wheeler-scratch/database_feature_selection_v13'
# filename_ = r'{}/{}'.format(save_path, file_names_[i][-14:])
# print(len(data_), filename_)
# with open(filename_, 'wb') as file:
#     pickle.dump(data_, file, protocol = pickle.HIGHEST_PROTOCOL)




['/users/terren/wheeler-scratch/data_feature_extraction_v2/2017_12_11.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2017_12_23.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2017_12_25.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2017_12_31.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_01_04.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_01_06.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_09.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_12.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_13.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_16.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_19.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_21.pkl', '/users/terren/wheeler-scratch/data_feature_extraction_v2/2018_12_23.pkl', '/users/terren/wheeler-s

error!
179 265 4
71
error!
180 265 4
86
error!
181 265 4
116
error!
182 265 4
273
error!
183 265 4
320
error!
184 265 4
126
error!
185 265 4
41
error!
186 265 4
66
error!
187 265 4
38
error!
188 265 4
41
error!
189 265 4
41
error!
190 265 4
41
error!
191 265 4
41
error!
192 265 4
41
error!
193 265 4
41
error!
194 265 4
41
error!
195 265 4
41
error!
196 265 4
41
error!
197 265 4
41
error!
198 265 4
41
error!
199 265 4
41
error!
200 265 4
41
error!
201 265 4
41
error!
202 265 4
41
error!
203 265 4
41
error!
204 265 4
41
error!
205 265 4
41
error!
206 265 4
41
error!
207 265 4
41
error!
208 265 4
41
error!
209 265 4
41
error!
210 265 4
41
error!
211 265 4
41
error!
212 265 4
41
error!
213 265 4
41
error!
214 265 4
41
error!
215 265 4
41
error!
216 265 4
41
error!
217 265 4
41
error!
218 265 4
41
error!
219 265 4
41
error!
220 265 4
41
error!
221 265 4
41
error!
222 265 4
41
error!
223 265 4
41
error!
224 265 4
41
error!
225 265 4
41
error!
226 265 4
41
error!
227 265 4
41
error!
228 265 4

error!
90 392 4
41
error!
91 392 4
41
error!
92 392 4
41
error!
93 392 4
41
error!
94 392 4
41
error!
95 392 4
41
error!
96 392 4
41
error!
97 392 4
41
error!
98 392 4
41
error!
99 392 4
41
error!
100 392 4
36
error!
101 392 4
41
error!
102 392 4
41
error!
103 392 4
41
error!
104 392 4
41
error!
105 392 4
41
error!
106 392 4
41
error!
107 392 4
41
error!
108 392 4
41
error!
109 392 4
41
error!
110 392 4
41
error!
111 392 4
41
error!
112 392 4
41
error!
113 392 4
41
error!
114 392 4
41
error!
115 392 4
41
error!
116 392 4
41
error!
117 392 4
41
error!
118 392 4
41
error!
119 392 4
41
error!
120 392 4
41
error!
121 392 4
41
error!
122 392 4
41
error!
123 392 4
41
error!
124 392 4
41
error!
125 392 4
41
error!
126 392 4
41
error!
127 392 4
41
error!
128 392 4
41
error!
129 392 4
41
error!
130 392 4
41
error!
131 392 4
41
error!
132 392 4
41
error!
133 392 4
41
error!
134 392 4
41
error!
135 392 4
41
error!
136 392 4
41
error!
137 392 4
43
error!
138 392 4
43
error!
139 392 4
43
error!
140

39
error!
162 423 4
34
error!
163 423 4
39
error!
164 423 4
39
error!
165 423 4
33
error!
166 423 4
39
error!
167 423 4
39
error!
168 423 4
39
error!
169 423 4
39
error!
170 423 4
39
error!
171 423 4
39
error!
172 423 4
39
error!
173 423 4
39
error!
174 423 4
39
error!
175 423 4
39
error!
176 423 4
39
error!
177 423 4
39
error!
178 423 4
39
error!
179 423 4
39
error!
180 423 4
39
error!
181 423 4
39
error!
182 423 4
39
error!
183 423 4
39
error!
184 423 4
49
error!
185 423 4
39
error!
186 423 4
43
error!
187 423 4
39
error!
188 423 4
43
error!
189 423 4
39
error!
190 423 4
39
error!
191 423 4
39
error!
192 423 4
39
error!
193 423 4
39
error!
194 423 4
39
error!
195 423 4
39
error!
196 423 4
39
error!
197 423 4
39
error!
198 423 4
39
error!
199 423 4
39
error!
200 423 4
39
error!
201 423 4
39
error!
202 423 4
39
error!
203 423 4
43
error!
204 423 4
43
error!
205 423 4
41
error!
206 423 4
43
error!
207 423 4
43
error!
208 423 4
43
error!
209 423 4
43
error!
210 423 4
51
error!
211 423 4


1284
error!
206 330 4
30
error!
207 330 4
29
error!
208 330 4
82
error!
209 330 4
29
error!
210 330 4
39
error!
211 330 4
41
error!
212 330 4
134
error!
213 330 4
41
error!
214 330 4
29
error!
215 330 4
29
error!
216 330 4
29
error!
217 330 4
128
error!
218 330 4
40
error!
219 330 4
29
error!
220 330 4
29
error!
221 330 4
599
error!
222 330 4
1107
error!
223 330 4
30
error!
224 330 4
41
error!
225 330 4
41
error!
226 330 4
41
error!
227 330 4
41
error!
228 330 4
39
error!
229 330 4
41
error!
230 330 4
29
error!
231 330 4
29
error!
232 330 4
31
error!
233 330 4
29
error!
234 330 4
51
error!
235 330 4
41
error!
236 330 4
41
error!
237 330 4
39
error!
238 330 4
41
error!
239 330 4
33
error!
240 330 4
29
error!
241 330 4
41
error!
242 330 4
29
error!
243 330 4
30
error!
244 330 4
39
error!
245 330 4
38
error!
246 330 4
41
error!
247 330 4
42
error!
248 330 4
39
error!
249 330 4
45
error!
250 330 4
31
error!
251 330 4
41
error!
252 330 4
42
error!
253 330 4
41
error!
254 330 4
1030
error!
2

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>